In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
from keras import metrics
from sklearn.model_selection import train_test_split

Using Theano backend.


In [3]:
data_lda = pickle.load(file=open('./mallet_doc_corpus.pickle', 'rb'))

In [4]:
data_original = pd.read_csv('./data_pd.csv')

In [5]:
#Augment Vector if Required
data_in = data_lda[:]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data_in, data_in, test_size=0.3, random_state=42)

In [7]:
batch_size = 100
original_dim = np.shape(data_in)[1]
latent_dim = 30
intermediate_dim = 70
epochs = 50
epsilon_std = 1.0


x =  Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var]) #Generate a sample 'z'

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)


# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

y = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, y)
vae.compile(optimizer='rmsprop', loss=None)

vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Output "custom_variational_layer_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_1" during training.


Train on 24005 samples, validate on 10289 samples
Epoch 1/50
23800/24005 [============================>.] - ETA: 0s - loss: 29.8198

ValueError: Input dimension mis-match. (input[0].shape[0] = 5, input[2].shape[0] = 100)
Apply node that caused the error: Elemwise{Composite{(i0 + (i1 * i2))}}(Elemwise{Add}[(0, 0)].0, Elemwise{Composite{exp((i0 * i1))}}[(0, 1)].0, Reshape{2}.0)
Toposort index: 46
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(5, 30), (5, 30), (100, 30)]
Inputs strides: [(120, 4), (120, 4), (120, 4)]
Inputs values: ['not shown', 'not shown', 'not shown']
Outputs clients: [[Dot22(Elemwise{Composite{(i0 + (i1 * i2))}}.0, dense_4/kernel), InplaceDimShuffle{1,0}(Elemwise{Composite{(i0 + (i1 * i2))}}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debu

In [ ]:
# build a model to project inputs on the latent space
encoder = Model(x, z_mean)
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)